<h1>Logistic Route Optimization - Illinois Idition</h1>

<h2>Project Scope</h2>
This notebook performs intercity delivery route optimization across key cities in the state of Illinois, with a central logistics hub in Chicago. It uses OpenStreetMap road network data to build a realistic driving graph and applies advanced algorithms to compute optimal delivery routes.

<h3>Goals</h3>
- Map delivery cities to the road network<br>
- Calculate travel distances between cities<br>
- Solve for the most efficient route using Google's OR-Tools<br>
- Visualize results using interactive maps<br>

Load Illinois cities dataset from CSV. If not found, generate it using the script.

In [5]:
import os
import sys

# Add 'src' to sys.path if it's not already there
src_path = os.path.abspath('../src/')
if src_path not in sys.path:
    sys.path.append(src_path)

In [6]:
import pandas as pd

csv_path = '../data/illinois_cities.csv'

try:
    df_cities = pd.read_csv(csv_path)
except FileNotFoundError:
    # Import the function
    from cities_dataset_generator import generate_illinois_cities_dataset

    # Generate the dataset
    generate_illinois_cities_dataset(csv_path)

    # Read it after creation
    df_cities = pd.read_csv(csv_path)

df_cities

,city,lat,lon
0,Chicago,41.8781,-87.6298
1,Aurora,41.7606,-88.3201
2,Rockford,42.2711,-89.0937
3,Naperville,41.7508,-88.1535
4,Joliet,41.5250,-88.0817
5,Springfield,39.7817,-89.6501
6,Peoria,40.6936,-89.5889
7,Champaign,40.1164,-88.2434


Download Illinois driving network with `osmnx`. If not found, generate it using the script.

In [8]:
import osmnx as ox

graph_path = '../data/illinois_graph.graphml'

try:
    G = ox.load_graphml(graph_path)
except FileNotFoundError:
    # Import the function
    from graphml_generator import generate_graphml

    # Genrate the graph
    generate_graphml(df_path='../data/illinois_cities.csv', file_path=graph_path)

    # Load after creation
    G = ox.load_graphml(graph_path)

Match cities to nearest road nodes

In [10]:
# Extract lat/lon from the dataframe
city_coords = list(zip(df_cities['lat'], df_cities['lon']))

# Get the nearest graph node for each city
city_node_ids = []
for lat, lon in city_coords:
    node = ox.distance.nearest_nodes(G, X=lon, Y=lat)
    city_node_ids.append(node)

df_cities['node_id'] = city_node_ids
df_cities

,city,lat,lon,node_id
0,Chicago,41.8781,-87.6298,261154311
1,Aurora,41.7606,-88.3201,235209482
2,Rockford,42.2711,-89.0937,237540096
3,Naperville,41.7508,-88.1535,237638509
4,Joliet,41.5250,-88.0817,237451309
5,Springfield,39.7817,-89.6501,1374889316
6,Peoria,40.6936,-89.5889,236827183
7,Champaign,40.1164,-88.2434,38035941
